# Indexing
## Text
### Preliminaries

In [ ]:
from pymongo import MongoClient, TEXT
from pprint import pprint

client = MongoClient()
restaurants = client.lab.restaurants
states = client.lab.states

### Exercise 1

In [ ]:
# Using regex
ex1 = restaurants.find({'name': {'$regex': '.*Poke.*'}}, {'_id': 0, 'name': 1, 'address': 1})
pprint(list(ex1))

In [ ]:
# Using text index
restaurants.create_index([('name', TEXT)])
ex11 = restaurants.find({'$text': {'$search': 'Poke'}}, {'_id': 0, 'name': 1, 'address': 1})
pprint(list(ex11))


### Exercise 2

In [ ]:
# First, find "Nolbu Restaurants". I will use the text index.
ex2_r = restaurants.find_one({'$text': {'$search': '\"Nolbu Restaurant\"'}})
ex2_res = states.find_one({
    'loc': {
        '$geoIntersects': {
            '$geometry': {
                'type': 'Point',
                'coordinates': ex2_r['address']['coord']
            }
        }
    }}, {'_id': 0, 'name': 1})

pprint(ex2_res)

### Exercise 3

In [ ]:
ex3_r = restaurants.find_one({'$text': {'$search': 'Ramen -Izakaya'}, 'cuisine': 'Korean'})
# pprint(ex3_r)
ex3_res = restaurants.find_one({
    'address.coord': {
        '$geoWithin': {
            '$centerSphere': [ex3_r['address']['coord'], 0.15 / 6378.1]}},
    'cuisine': 'Japanese'})
pprint(ex3_res)


In [ ]:
restaurants.find_one({
    'address.coord': {
        '$nearSphere': {
            '$geometry': {
                'type': "Point",
                'coordinates': ex3_r['address']['coord']
            },
            '$maxDistance': 150
        }
    },
    'cuisine': 'Japanese'
})